<a href="https://colab.research.google.com/github/SarmadAslam/Data-Science/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install pdfplumber pytesseract pdf2image sentence-transformers faiss-cpu transformers fuzzywuzzy python-Levenshtein streamlit pyngrok requests pillow torch

In [2]:
# # 🛠️ 1. System-Level Dependencies (for Camelot, Tesseract, Tabula)
# !apt-get update
# !apt-get install -y ghostscript python3-tk tesseract-ocr openjdk-11-jre

# # 🧠 2. Download spaCy language model
# !python -m spacy download en_core_web_sm

# # 📦 3. Python Dependencies
# !pip install \
#     google-generativeai \
#     pandas \
#     spacy \
#     camelot-py[cv] \
#     tabula-py \
#     pdfminer.six \
#     PyMuPDF \
#     scikit-learn \
#     phonenumbers \
#     email-validator \
#     backoff


In [3]:
# !apt-get update
# !apt-get install -y tesseract-ocr
# !apt-get install -y poppler-utils

In [4]:
!ngrok authtoken 30mDYr0DqRNWeehRC3sQ5qDqnJg_2EoFLvkY2GTo82ydKvRCA

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
# script_content = '''
# import pdfplumber
# import pytesseract
# from PIL import Image
# import pdf2image
# import re
# import numpy as np
# from sentence_transformers import SentenceTransformer
# import faiss
# import streamlit as st
# import requests
# import json
# import os
# import logging
# import traceback
# import google.generativeai as genai
# from typing import List, Dict, Any, Optional, Tuple
# import pandas as pd
# from dataclasses import dataclass, field
# import spacy
# from fuzzywuzzy import fuzz
# import time
# from collections import defaultdict
# import hashlib
# from datetime import datetime
# import camelot
# import tabula
# from pdfminer.high_level import extract_text
# from pdfminer.layout import LAParams
# import fitz  # PyMuPDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import phonenumbers
# import email_validator
# from urllib.parse import urlparse
# import concurrent.futures
# from functools import lru_cache
# import backoff

# # Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
# )
# logger = logging.getLogger(__name__)

# # API Configuration
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
# DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY", "sk-640564380f8645a0aadb6168fc70a96a")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

# # Configure Gemini
# if GEMINI_API_KEY:
#     genai.configure(api_key=GEMINI_API_KEY)

# @dataclass
# class Contact:
#     """Enhanced contact information with metadata"""
#     name: str = ""
#     business_name: str = ""
#     email: str = ""
#     phone: str = ""
#     address: str = ""
#     website: str = ""
#     title: str = ""
#     department: str = ""
#     linkedin: str = ""
#     fax: str = ""
#     mobile: str = ""
#     confidence_score: float = 0.0
#     source_context: str = ""
#     extraction_method: str = ""
#     page_number: int = 0
#     position_in_doc: Tuple[float, float] = (0.0, 0.0)
#     metadata: Dict[str, Any] = field(default_factory=dict)

#     def __hash__(self):
#         return hash((self.email, self.phone, self.name))

#     def to_dict(self):
#         return {k: v for k, v in self.__dict__.items() if v}

# class DocumentAnalyzer:
#     """Analyzes PDF structure and content type"""

#     def __init__(self):
#         self.patterns = {
#             'table': r'(?i)(name|email|phone|contact|address|website|fax|mobile)',
#             'business_card': r'(?i)(tel|mob|email|www\.|@)',
#             'directory': r'(?i)(member|directory|list|roster|association)',
#             'resume': r'(?i)(experience|education|skills|objective)',
#             'form': r'(?i)(form|application|registration)',
#         }

#     def analyze_document(self, text: str, pages: List[Any]) -> Dict[str, Any]:
#         """Analyze document structure and type"""
#         analysis = {
#             'type': 'unknown',
#             'has_tables': False,
#             'is_scanned': False,
#             'language': 'en',
#             'contact_density': 0,
#             'structure_type': 'unstructured',
#             'recommended_methods': []
#         }

#         # Check if scanned
#         if not text.strip() and pages:
#             analysis['is_scanned'] = True

#         # Detect document type
#         text_lower = text.lower()
#         for doc_type, pattern in self.patterns.items():
#             if len(re.findall(pattern, text_lower)) > 3:
#                 analysis['type'] = doc_type
#                 break

#         # Calculate contact density
#         contact_indicators = len(re.findall(r'(?i)(email|phone|tel|mobile|@)', text))
#         analysis['contact_density'] = contact_indicators / max(len(text.split()), 1)

#         # Recommend extraction methods
#         if analysis['type'] == 'table' or 'table' in text_lower:
#             analysis['recommended_methods'] = ['table_extraction', 'structured_parsing']
#             analysis['structure_type'] = 'tabular'
#         elif analysis['is_scanned']:
#             analysis['recommended_methods'] = ['ocr_enhanced', 'layout_analysis']
#         elif analysis['contact_density'] > 0.01:
#             analysis['recommended_methods'] = ['pattern_matching', 'ai_extraction']
#             analysis['structure_type'] = 'semi_structured'
#         else:
#             analysis['recommended_methods'] = ['ai_extraction', 'nlp_parsing']

#         return analysis

# class IntelligentPDFExtractor:
#     """Multi-method PDF extraction with intelligent fallbacks"""

#     def __init__(self):
#         self.analyzer = DocumentAnalyzer()

#     def extract_with_pdfplumber(self, pdf_path: str) -> Tuple[str, List[Dict]]:
#         """Extract using pdfplumber with enhanced parsing"""
#         text = ""
#         tables = []

#         try:
#             with pdfplumber.open(pdf_path) as pdf:
#                 for page_num, page in enumerate(pdf.pages):
#                     # Extract text
#                     page_text = page.extract_text() or ""
#                     text += f"\n--- PAGE {page_num + 1} ---\n{page_text}\n"

#                     # Extract tables
#                     page_tables = page.extract_tables()
#                     if page_tables:
#                         for table in page_tables:
#                             if table:
#                                 tables.append({
#                                     'page': page_num + 1,
#                                     'data': table
#                                 })
#         except Exception as e:
#             logger.error(f"PDFPlumber extraction failed: {e}")

#         return text, tables

#     def extract_with_camelot(self, pdf_path: str) -> List[pd.DataFrame]:
#         """Extract tables using Camelot"""
#         try:
#             tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice')
#             if not tables:
#                 tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
#             return [table.df for table in tables]
#         except Exception as e:
#             logger.error(f"Camelot extraction failed: {e}")
#             return []

#     def extract_with_tabula(self, pdf_path: str) -> List[pd.DataFrame]:
#         """Extract tables using Tabula"""
#         try:
#             return tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
#         except Exception as e:
#             logger.error(f"Tabula extraction failed: {e}")
#             return []

#     def extract_with_pdfminer(self, pdf_path: str) -> str:
#         """Extract with PDFMiner for better layout preservation"""
#         try:
#             laparams = LAParams(
#                 line_overlap=0.5,
#                 char_margin=2.0,
#                 word_margin=0.1,
#                 boxes_flow=0.5
#             )
#             return extract_text(pdf_path, laparams=laparams)
#         except Exception as e:
#             logger.error(f"PDFMiner extraction failed: {e}")
#             return ""

#     def extract_with_pymupdf(self, pdf_path: str) -> Tuple[str, List[Dict]]:
#         """Extract using PyMuPDF with layout analysis"""
#         text = ""
#         metadata = []

#         try:
#             doc = fitz.open(pdf_path)
#             for page_num, page in enumerate(doc):
#                 # Extract text with position
#                 page_text = page.get_text()
#                 text += f"\n--- PAGE {page_num + 1} ---\n{page_text}\n"

#                 # Extract text blocks with positions
#                 blocks = page.get_text("dict")
#                 for block in blocks["blocks"]:
#                     if "lines" in block:
#                         for line in block["lines"]:
#                             for span in line["spans"]:
#                                 metadata.append({
#                                     'page': page_num + 1,
#                                     'text': span['text'],
#                                     'bbox': span['bbox'],
#                                     'font': span.get('font', ''),
#                                     'size': span.get('size', 0)
#                                 })
#             doc.close()
#         except Exception as e:
#             logger.error(f"PyMuPDF extraction failed: {e}")

#         return text, metadata

#     def enhanced_ocr_extraction(self, pdf_path: str) -> str:
#         """Enhanced OCR with preprocessing"""
#         text = ""

#         try:
#             images = pdf2image.convert_from_path(pdf_path, dpi=300)

#             for i, img in enumerate(images):
#                 # Preprocess image
#                 img = img.convert('L')  # Grayscale
#                 img_array = np.array(img)

#                 # Adaptive thresholding
#                 from cv2 import adaptiveThreshold, ADAPTIVE_THRESH_GAUSSIAN_C, THRESH_BINARY
#                 img_array = adaptiveThreshold(
#                     img_array, 255,
#                     ADAPTIVE_THRESH_GAUSSIAN_C,
#                     THRESH_BINARY, 11, 2
#                 )

#                 # OCR with multiple configurations
#                 configs = [
#                     '--psm 6',  # Uniform block of text
#                     '--psm 4',  # Single column
#                     '--psm 3',  # Fully automatic
#                 ]

#                 best_text = ""
#                 max_conf = 0

#                 for config in configs:
#                     try:
#                         data = pytesseract.image_to_data(
#                             img_array,
#                             config=config,
#                             output_type=pytesseract.Output.DICT
#                         )

#                         conf = np.mean([int(c) for c in data['conf'] if int(c) > 0])
#                         page_text = " ".join([
#                             data['text'][i]
#                             for i in range(len(data['text']))
#                             if int(data['conf'][i]) > 30
#                         ])

#                         if conf > max_conf:
#                             max_conf = conf
#                             best_text = page_text
#                     except:
#                         continue

#                 text += f"\n--- PAGE {i + 1} ---\n{best_text}\n"

#         except Exception as e:
#             logger.error(f"Enhanced OCR failed: {e}")

#         return text

#     def extract_all_methods(self, pdf_path: str) -> Dict[str, Any]:
#         """Extract using all methods and combine results"""
#         results = {
#             'text': '',
#             'tables': [],
#             'metadata': [],
#             'extraction_methods': []
#         }

#         # Analyze document first
#         with pdfplumber.open(pdf_path) as pdf:
#             sample_text = ""
#             for i, page in enumerate(pdf.pages[:3]):  # Sample first 3 pages
#                 sample_text += (page.extract_text() or "")

#             analysis = self.analyzer.analyze_document(sample_text, pdf.pages)

#         # Apply recommended methods
#         if 'table_extraction' in analysis['recommended_methods']:
#             # Try table extraction methods
#             camelot_tables = self.extract_with_camelot(pdf_path)
#             if camelot_tables:
#                 results['tables'].extend([{
#                     'method': 'camelot',
#                     'data': table
#                 } for table in camelot_tables])
#                 results['extraction_methods'].append('camelot')

#             tabula_tables = self.extract_with_tabula(pdf_path)
#             if tabula_tables:
#                 results['tables'].extend([{
#                     'method': 'tabula',
#                     'data': table
#                 } for table in tabula_tables])
#                 results['extraction_methods'].append('tabula')

#         # Extract text with multiple methods
#         text1, plumber_tables = self.extract_with_pdfplumber(pdf_path)
#         if text1:
#             results['text'] = text1
#             results['extraction_methods'].append('pdfplumber')
#             if plumber_tables:
#                 results['tables'].extend([{
#                     'method': 'pdfplumber',
#                     'data': table
#                 } for table in plumber_tables])

#         # Try PDFMiner for better layout
#         if not results['text'] or analysis['structure_type'] == 'semi_structured':
#             text2 = self.extract_with_pdfminer(pdf_path)
#             if text2 and len(text2) > len(results['text']):
#                 results['text'] = text2
#                 results['extraction_methods'].append('pdfminer')

#         # Try PyMuPDF for metadata
#         text3, metadata = self.extract_with_pymupdf(pdf_path)
#         if metadata:
#             results['metadata'] = metadata
#             results['extraction_methods'].append('pymupdf')
#             if not results['text']:
#                 results['text'] = text3

#         # OCR as last resort or if scanned
#         if not results['text'].strip() or analysis['is_scanned']:
#             ocr_text = self.enhanced_ocr_extraction(pdf_path)
#             if ocr_text:
#                 results['text'] = ocr_text
#                 results['extraction_methods'].append('enhanced_ocr')

#         results['analysis'] = analysis
#         return results

# class IntelligentTextChunker:
#     """Context-aware text chunking that preserves contact boundaries"""

#     def __init__(self, nlp_model=None):
#         self.nlp = nlp_model
#         self.contact_patterns = [
#             r'(?i)(?:name|contact|person).*?(?:email|phone|tel|mobile|address)',
#             r'(?i)(?:mr|mrs|ms|dr|prof)\.?\s+[A-Z][a-z]+.*?(?:@|\d{3,})',
#             r'[A-Z][a-z]+\s+[A-Z][a-z]+.*?(?:\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b)',
#         ]

#     def identify_contact_boundaries(self, text: str) -> List[Tuple[int, int]]:
#         """Identify potential contact information boundaries"""
#         boundaries = []

#         # Find pattern-based boundaries
#         for pattern in self.contact_patterns:
#             for match in re.finditer(pattern, text, re.DOTALL):
#                 boundaries.append((match.start(), match.end()))

#         # Merge overlapping boundaries
#         boundaries.sort()
#         merged = []
#         for start, end in boundaries:
#             if merged and start <= merged[-1][1] + 50:  # Allow 50 char gap
#                 merged[-1] = (merged[-1][0], max(merged[-1][1], end))
#             else:
#                 merged.append((start, end))

#         return merged

#     def chunk_by_structure(self, text: str, tables: List[Dict]) -> List[Dict[str, Any]]:
#         """Create chunks based on document structure"""
#         chunks = []

#         # Handle tables separately
#         for table in tables:
#             if isinstance(table.get('data'), pd.DataFrame):
#                 chunks.append({
#                     'type': 'table',
#                     'content': table['data'].to_string(),
#                     'data': table['data'],
#                     'method': table.get('method', 'unknown')
#                 })
#             elif isinstance(table.get('data'), list):
#                 chunks.append({
#                     'type': 'table',
#                     'content': '\n'.join(['\t'.join(map(str, row)) for row in table['data'] if row]),
#                     'data': table['data'],
#                     'method': table.get('method', 'unknown')
#                 })

#         # Handle text with smart chunking
#         if text:
#             # Split by pages first
#             pages = re.split(r'--- PAGE \d+ ---', text)

#             for page_num, page_text in enumerate(pages):
#                 if not page_text.strip():
#                     continue

#                 # Find contact boundaries in this page
#                 boundaries = self.identify_contact_boundaries(page_text)

#                 if boundaries:
#                     # Chunk based on boundaries
#                     last_end = 0
#                     for start, end in boundaries:
#                         # Include context before
#                         context_start = max(0, start - 100)
#                         context_end = min(len(page_text), end + 100)

#                         chunk_text = page_text[context_start:context_end]
#                         chunks.append({
#                             'type': 'contact_region',
#                             'content': chunk_text,
#                             'page': page_num + 1,
#                             'confidence': 0.8
#                         })
#                         last_end = end
#                 else:
#                     # Smart paragraph chunking
#                     paragraphs = re.split(r'\n\s*\n', page_text)
#                     current_chunk = ""

#                     for para in paragraphs:
#                         if len(current_chunk) + len(para) > 1500:
#                             if current_chunk:
#                                 chunks.append({
#                                     'type': 'text',
#                                     'content': current_chunk,
#                                     'page': page_num + 1
#                                 })
#                             current_chunk = para
#                         else:
#                             current_chunk += "\n\n" + para

#                     if current_chunk:
#                         chunks.append({
#                             'type': 'text',
#                             'content': current_chunk,
#                             'page': page_num + 1
#                         })

#         return chunks

#     def semantic_chunking(self, text: str, max_chunk_size: int = 1000) -> List[str]:
#         """Chunk based on semantic boundaries using NLP"""
#         if not self.nlp:
#             return self.simple_chunking(text, max_chunk_size)

#         try:
#             doc = self.nlp(text[:1000000])  # Limit for performance
#             chunks = []
#             current_chunk = []
#             current_size = 0

#             for sent in doc.sents:
#                 sent_text = sent.text.strip()
#                 sent_size = len(sent_text)

#                 # Check if this sentence likely contains contact info
#                 has_contact = any(
#                     ent.label_ in ['PERSON', 'ORG', 'EMAIL', 'PHONE']
#                     for ent in sent.ents
#                 )

#                 if current_size + sent_size > max_chunk_size and current_chunk:
#                     # Don't break if we're in the middle of contact info
#                     if not has_contact or current_size > max_chunk_size * 1.5:
#                         chunks.append(' '.join(current_chunk))
#                         current_chunk = [sent_text]
#                         current_size = sent_size
#                     else:
#                         current_chunk.append(sent_text)
#                         current_size += sent_size
#                 else:
#                     current_chunk.append(sent_text)
#                     current_size += sent_size

#             if current_chunk:
#                 chunks.append(' '.join(current_chunk))

#             return chunks

#         except Exception as e:
#             logger.error(f"Semantic chunking failed: {e}")
#             return self.simple_chunking(text, max_chunk_size)

#     def simple_chunking(self, text: str, chunk_size: int = 1000) -> List[str]:
#         """Fallback simple chunking with overlap"""
#         chunks = []
#         sentences = re.split(r'[.!?\n]+', text)
#         current_chunk = ""

#         for sentence in sentences:
#             if len(current_chunk) + len(sentence) > chunk_size:
#                 if current_chunk:
#                     chunks.append(current_chunk)
#                 current_chunk = sentence
#             else:
#                 current_chunk += " " + sentence

#         if current_chunk:
#             chunks.append(current_chunk)

#         return chunks

# class AdaptivePromptEngine:
#     """Generates context-aware prompts based on document analysis"""

#     def __init__(self):
#         self.prompt_templates = {
#             'table': """
# You are analyzing a TABULAR document with structured contact information.

# EXTRACTION RULES:
# 1. Each row likely represents one contact
# 2. Column headers indicate data types (Name, Email, Phone, etc.)
# 3. Maintain row integrity - don't mix data between rows
# 4. Handle multi-line cells and merged cells correctly
# 5. Extract ALL available fields including: Name, Title, Company, Email, Phone, Mobile, Fax, Address, Website

# TABLE DATA:
# {content}

# Return ONLY a JSON array with complete contact objects. Include confidence scores based on data completeness.
# """,

#             'directory': """
# You are extracting from a BUSINESS DIRECTORY or MEMBERSHIP LIST.

# PATTERN RECOGNITION:
# 1. Entries may follow patterns like: "Company Name - Contact Person - Phone/Email"
# 2. Look for section headers, member numbers, or categories
# 3. Addresses often span multiple lines
# 4. Some entries may have multiple contacts per organization
# 5. Preserve hierarchical relationships (company -> employees)

# DIRECTORY CONTENT:
# {content}

# Extract all contacts maintaining their organizational relationships. Return JSON array with nested structures if needed.
# """,

#             'business_card': """
# You are extracting from BUSINESS CARD style layouts.

# EXTRACTION FOCUS:
# 1. Primary contact is usually the card owner
# 2. Look for job titles near names
# 3. Company logos/names are typically prominent
# 4. Multiple phone numbers may be listed (mobile, office, fax)
# 5. Social media handles and websites are common

# CONTENT:
# {content}

# Extract complete contact information with high confidence for clearly identified fields.
# """,

#             'unstructured': """
# You are an expert contact extraction specialist analyzing an unstructured document.

# ADVANCED EXTRACTION TECHNIQUES:
# 1. Use contextual clues to associate information
# 2. Names often appear before or after titles
# 3. Email patterns: [name]@[domain].[tld]
# 4. Phone patterns vary by country - extract all formats
# 5. Websites may not include http://
# 6. LinkedIn URLs often contain profile names
# 7. Addresses may be incomplete - extract what's available

# CRITICAL RULES:
# - Each contact MUST have at least email OR phone to be valid
# - Associate information based on proximity and context
# - Don't create contacts from partial information
# - Include confidence scores (0.0-1.0) based on:
#   * Information completeness
#   * Clear associations
#   * Data quality

# DOCUMENT CONTENT:
# {content}

# METADATA:
# {metadata}

# Return a JSON array of contact objects. Be conservative - only extract contacts you're confident about.
# """,

#             'mixed': """
# You are analyzing a document with MIXED FORMATS containing various contact information.

# MULTI-FORMAT HANDLING:
# 1. Document may contain tables, paragraphs, lists, and cards
# 2. Use format-appropriate extraction for each section
# 3. Cross-reference information when the same contact appears multiple times
# 4. Handle variations in name spelling and formatting
# 5. Merge duplicate entries intelligently

# SECTION: {section_type}
# {content}

# Extract all valid contacts from this section. Consider the section type when applying extraction rules.
# """
#         }

#     def generate_prompt(self, chunk: Dict[str, Any], doc_analysis: Dict[str, Any]) -> str:
#         """Generate optimal prompt based on content analysis"""

#         # Select base template
#         doc_type = doc_analysis.get('type', 'unstructured')
#         chunk_type = chunk.get('type', 'text')

#         if chunk_type == 'table':
#             template = self.prompt_templates['table']
#         elif doc_type in self.prompt_templates:
#             template = self.prompt_templates[doc_type]
#         else:
#             template = self.prompt_templates['unstructured']

#         # Enhance prompt with specific instructions
#         content = chunk.get('content', '')
#         metadata = {
#             'chunk_type': chunk_type,
#             'page': chunk.get('page', 'unknown'),
#             'extraction_method': chunk.get('method', 'unknown'),
#             'document_type': doc_type,
#             'confidence_hint': chunk.get('confidence', 0.5)
#         }

#         # Add dynamic instructions based on content analysis
#         if '@' in content and '.com' in content:
#             extra_instructions = "\nPay special attention to email addresses and their associated names."
#         elif re.search(r'\+?\d{10,}', content):
#             extra_instructions = "\nMultiple phone formats detected. Extract all variations."
#         elif 'ltd' in content.lower() or 'inc' in content.lower() or 'corp' in content.lower():
#             extra_instructions = "\nCorporate entities detected. Distinguish between company and person names."
#         else:
#             extra_instructions = ""

#         # Fill template
#         prompt = template.format(
#             content=content,
#             metadata=json.dumps(metadata, indent=2),
#             section_type=chunk_type
#         )

#         return prompt + extra_instructions

# class SmartAPIManager:
#     """Manages multiple APIs with intelligent fallback and rate limiting"""

#     def __init__(self):
#         self.api_configs = {
#             'gemini': {
#                 'enabled': bool(GEMINI_API_KEY),
#                 'rate_limit': 60,  # requests per minute
#                 'timeout': 30,
#                 'priority': 1
#             },
#             'deepseek': {
#                 'enabled': bool(DEEPSEEK_API_KEY),
#                 'rate_limit': 30,
#                 'timeout': 30,
#                 'priority': 2
#             },
#             'openai': {
#                 'enabled': bool(OPENAI_API_KEY),
#                 'rate_limit': 60,
#                 'timeout': 30,
#                 'priority': 3
#             }
#         }

#         self.request_history = defaultdict(list)
#         self.prompt_engine = AdaptivePromptEngine()

#     def _check_rate_limit(self, api_name: str) -> bool:
#         """Check if API rate limit allows request"""
#         config = self.api_configs[api_name]
#         now = time.time()

#         # Clean old requests
#         self.request_history[api_name] = [
#             t for t in self.request_history[api_name]
#             if now - t < 60
#         ]

#         return len(self.request_history[api_name]) < config['rate_limit']

#     @backoff.on_exception(
#         backoff.expo,
#         (requests.exceptions.RequestException, Exception),
#         max_tries=3
#     )
#     def _call_gemini(self, prompt: str) -> Optional[List[Dict]]:
#         """Call Gemini API with retry logic"""
#         if not self._check_rate_limit('gemini'):
#             logger.warning("Gemini rate limit reached")
#             return None

#         try:
#             model = genai.GenerativeModel('gemini-1.5-flash')

#             # Enhanced prompt for better JSON extraction
#             enhanced_prompt = prompt + """

# IMPORTANT: Return ONLY valid JSON array. No explanations or markdown.
# Example format:
# [
#   {
#     "name": "John Smith",
#     "email": "john@example.com",
#     "phone": "+1234567890",
#     "confidence_score": 0.95
#   }
# ]
# """

#             response = model.generate_content(enhanced_prompt)
#             self.request_history['gemini'].append(time.time())

#             # Extract JSON from response
#             response_text = response.text.strip()

#             # Try to find JSON array in response
#             json_match = re.search(r'\[[\s\S]*\]', response_text)
#             if json_match:
#                 contacts_data = json.loads(json_match.group())
#                 return contacts_data

#             return None

#         except Exception as e:
#             logger.error(f"Gemini API error: {e}")
#             return None

#     @backoff.on_exception(
#         backoff.expo,
#         requests.exceptions.RequestException,
#         max_tries=3
#     )
#     def _call_deepseek(self, prompt: str) -> Optional[List[Dict]]:
#         """Call DeepSeek API with retry logic"""
#         if not self._check_rate_limit('deepseek'):
#             logger.warning("DeepSeek rate limit reached")
#             return None

#         try:
#             headers = {
#                 "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
#                 "Content-Type": "application/json"
#             }

#             data = {
#                 "model": "deepseek-chat",
#                 "messages": [{
#                     "role": "system",
#                     "content": "You are a contact extraction expert. Always return valid JSON arrays only."
#                 }, {
#                     "role": "user",
#                     "content": prompt
#                 }],
#                 "temperature": 0.1,
#                 "max_tokens": 2000
#             }

#             response = requests.post(
#                 "https://api.deepseek.com/v1/chat/completions",
#                 headers=headers,
#                 json=data,
#                 timeout=30
#             )

#             if response.status_code == 200:
#                 self.request_history['deepseek'].append(time.time())
#                 result = response.json()
#                 response_text = result["choices"][0]["message"]["content"]

#                 # Extract JSON
#                 json_match = re.search(r'\[[\s\S]*\]', response_text)
#                 if json_match:
#                     return json.loads(json_match.group())

#             return None

#         except Exception as e:
#             logger.error(f"DeepSeek API error: {e}")
#             return None

#     def extract_contacts_from_chunk(
#         self,
#         chunk: Dict[str, Any],
#         doc_analysis: Dict[str, Any]
#     ) -> List[Contact]:
#         """Extract contacts using available APIs with fallback"""

#         # Generate optimal prompt
#         prompt = self.prompt_engine.generate_prompt(chunk, doc_analysis)

#         # Try APIs in priority order
#         apis = sorted(
#             [(name, cfg) for name, cfg in self.api_configs.items() if cfg['enabled']],
#             key=lambda x: x[1]['priority']
#         )

#         for api_name, config in apis:
#             logger.info(f"Trying {api_name} API for extraction")

#             contacts_data = None
#             if api_name == 'gemini':
#                 contacts_data = self._call_gemini(prompt)
#             elif api_name == 'deepseek':
#                 contacts_data = self._call_deepseek(prompt)
#             # Add more APIs as needed

#             if contacts_data:
#                 # Convert to Contact objects
#                 contacts = []
#                 for contact_dict in contacts_data:
#                     if isinstance(contact_dict, dict):
#                         contact = Contact(
#                             name=contact_dict.get('name', '').strip(),
#                             business_name=contact_dict.get('business_name', '').strip(),
#                             email=contact_dict.get('email', '').strip(),
#                             phone=contact_dict.get('phone', '').strip(),
#                             address=contact_dict.get('address', '').strip(),
#                             website=contact_dict.get('website', '').strip(),
#                             title=contact_dict.get('title', '').strip(),
#                             department=contact_dict.get('department', '').strip(),
#                             linkedin=contact_dict.get('linkedin', '').strip(),
#                             fax=contact_dict.get('fax', '').strip(),
#                             mobile=contact_dict.get('mobile', '').strip(),
#                             confidence_score=float(contact_dict.get('confidence_score', 0.7)),
#                             source_context=chunk.get('content', '')[:200],
#                             extraction_method=f"{api_name}_{chunk.get('type', 'unknown')}",
#                             page_number=chunk.get('page', 0)
#                         )
#                         contacts.append(contact)

#                 return contacts

#         logger.warning("All API extractions failed")
#         return []

# class AdvancedValidator:
#     """Multi-level validation with confidence scoring"""

#     def __init__(self):
#         self.email_validator = None
#         try:
#             import email_validator
#             self.email_validator = email_validator
#         except:
#             pass

#     def validate_email(self, email: str) -> Tuple[bool, float]:
#         """Validate email with confidence score"""
#         if not email:
#             return False, 0.0

#         # Basic pattern check
#         basic_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
#         if not re.match(basic_pattern, email):
#             return False, 0.0

#         confidence = 0.7

#         # Advanced validation if available
#         if self.email_validator:
#             try:
#                 validation = self.email_validator.validate_email(email)
#                 email = validation.email
#                 confidence = 0.95
#             except:
#                 confidence = 0.5

#         # Check for common patterns
#         if any(domain in email.lower() for domain in ['gmail.', 'yahoo.', 'hotmail.', 'outlook.']):
#             confidence = min(1.0, confidence + 0.1)

#         # Check for suspicious patterns
#         if email.count('@') > 1 or '..' in email:
#             confidence *= 0.5

#         return True, confidence

#     def validate_phone(self, phone: str) -> Tuple[bool, float, str]:
#         """Validate and normalize phone with confidence"""
#         if not phone:
#             return False, 0.0, ""

#         # Remove common formatting
#         cleaned = re.sub(r'[^\d+]', '', phone)

#         if len(cleaned) < 7:
#             return False, 0.0, ""

#         confidence = 0.6
#         normalized = cleaned

#         try:
#             # Try to parse with phonenumbers library
#             import phonenumbers

#             # Try with different country codes
#             parsed = None
#             for country in ['US', 'GB', 'PK', 'IN', None]:
#                 try:
#                     parsed = phonenumbers.parse(phone, country)
#                     if phonenumbers.is_valid_number(parsed):
#                         normalized = phonenumbers.format_number(
#                             parsed,
#                             phonenumbers.PhoneNumberFormat.INTERNATIONAL
#                         )
#                         confidence = 0.95
#                         break
#                 except:
#                     continue

#             if not parsed:
#                 # Basic validation
#                 if 10 <= len(cleaned) <= 15:
#                     confidence = 0.7
#                     normalized = cleaned
#                 else:
#                     confidence = 0.4

#         except ImportError:
#             # Fallback validation
#             if re.match(r'^\+?\d{10,15}$', cleaned):
#                 confidence = 0.7
#                 normalized = cleaned

#         return True, confidence, normalized

#     def validate_website(self, url: str) -> Tuple[bool, float, str]:
#         """Validate and normalize website URL"""
#         if not url:
#             return False, 0.0, ""

#         # Add protocol if missing
#         if not url.startswith(('http://', 'https://')):
#             url = 'https://' + url

#         try:
#             parsed = urlparse(url)
#             if parsed.netloc:
#                 # Check for common TLDs
#                 tld = parsed.netloc.split('.')[-1].lower()
#                 common_tlds = ['com', 'org', 'net', 'edu', 'gov', 'io', 'co', 'uk', 'de', 'fr']

#                 confidence = 0.8 if tld in common_tlds else 0.6

#                 # Check for suspicious patterns
#                 if parsed.netloc.count('.') > 3:
#                     confidence *= 0.7

#                 return True, confidence, url
#         except:
#             pass

#         return False, 0.0, ""

#     def validate_name(self, name: str) -> Tuple[bool, float]:
#         """Validate name with confidence"""
#         if not name or len(name) < 2:
#             return False, 0.0

#         # Check for common patterns
#         confidence = 0.7

#         # Has first and last name
#         if len(name.split()) >= 2:
#             confidence += 0.1

#         # Contains only valid characters
#         if re.match(r'^[A-Za-z\s\'-\.]+$', name):
#             confidence += 0.1

#         # Not too long
#         if len(name) > 50:
#             confidence *= 0.7

#         # Has proper capitalization
#         if name[0].isupper():
#             confidence += 0.05

#         return True, min(1.0, confidence)

#     def calculate_overall_confidence(self, contact: Contact) -> float:
#         """Calculate overall confidence score for contact"""
#         scores = []
#         weights = []

#         # Email validation
#         if contact.email:
#             valid, conf = self.validate_email(contact.email)
#             if valid:
#                 scores.append(conf)
#                 weights.append(2.0)  # Email is important

#         # Phone validation
#         if contact.phone:
#             valid, conf, _ = self.validate_phone(contact.phone)
#             if valid:
#                 scores.append(conf)
#                 weights.append(2.0)  # Phone is important

#         # Name validation
#         if contact.name:
#             valid, conf = self.validate_name(contact.name)
#             if valid:
#                 scores.append(conf)
#                 weights.append(1.5)

#         # Website validation
#         if contact.website:
#             valid, conf, _ = self.validate_website(contact.website)
#             if valid:
#                 scores.append(conf)
#                 weights.append(1.0)

#         # Business name
#         if contact.business_name:
#             scores.append(0.8)
#             weights.append(1.0)

#         # Must have email or phone
#         if not contact.email and not contact.phone:
#             return 0.0

#         # Calculate weighted average
#         if scores:
#             weighted_sum = sum(s * w for s, w in zip(scores, weights))
#             total_weight = sum(weights)
#             base_confidence = weighted_sum / total_weight

#             # Boost for completeness
#             fields_filled = sum([
#                 bool(contact.name),
#                 bool(contact.email),
#                 bool(contact.phone),
#                 bool(contact.business_name),
#                 bool(contact.address),
#                 bool(contact.website),
#                 bool(contact.title)
#             ])

#             completeness_boost = fields_filled / 7 * 0.2

#             return min(1.0, base_confidence + completeness_boost)

#         return contact.confidence_score

# class IntelligentDeduplicator:
#     """Advanced deduplication using multiple similarity metrics"""

#     def __init__(self):
#         self.validators = AdvancedValidator()

#     def calculate_similarity(self, contact1: Contact, contact2: Contact) -> float:
#         """Calculate similarity between two contacts"""
#         scores = []
#         weights = []

#         # Email similarity (exact match only)
#         if contact1.email and contact2.email:
#             if contact1.email.lower() == contact2.email.lower():
#                 scores.append(1.0)
#                 weights.append(3.0)
#             else:
#                 scores.append(0.0)
#                 weights.append(3.0)

#         # Phone similarity
#         if contact1.phone and contact2.phone:
#             # Normalize phones
#             _, _, norm1 = self.validators.validate_phone(contact1.phone)
#             _, _, norm2 = self.validators.validate_phone(contact2.phone)

#             if norm1 and norm2:
#                 # Extract digits only
#                 digits1 = re.sub(r'\D', '', norm1)
#                 digits2 = re.sub(r'\D', '', norm2)

#                 if digits1 == digits2:
#                     scores.append(1.0)
#                 elif digits1.endswith(digits2[-7:]) or digits2.endswith(digits1[-7:]):
#                     scores.append(0.8)
#                 else:
#                     scores.append(0.0)
#                 weights.append(2.5)

#         # Name similarity
#         if contact1.name and contact2.name:
#             # Use fuzzy matching
#             name_score = fuzz.token_sort_ratio(
#                 contact1.name.lower(),
#                 contact2.name.lower()
#             ) / 100.0

#             scores.append(name_score)
#             weights.append(1.5)

#         # Business name similarity
#         if contact1.business_name and contact2.business_name:
#             biz_score = fuzz.token_sort_ratio(
#                 contact1.business_name.lower(),
#                 contact2.business_name.lower()
#             ) / 100.0

#             scores.append(biz_score)
#             weights.append(1.0)

#         # Calculate weighted average
#         if scores:
#             weighted_sum = sum(s * w for s, w in zip(scores, weights))
#             total_weight = sum(weights)
#             return weighted_sum / total_weight

#         return 0.0

#     def merge_contacts(self, contact1: Contact, contact2: Contact) -> Contact:
#         """Intelligently merge two similar contacts"""
#         # Use the contact with higher confidence as base
#         if contact1.confidence_score >= contact2.confidence_score:
#             merged = Contact(**contact1.__dict__)
#             donor = contact2
#         else:
#             merged = Contact(**contact2.__dict__)
#             donor = contact1

#         # Fill in missing fields
#         for field in ['name', 'business_name', 'email', 'phone', 'address',
#                      'website', 'title', 'department', 'linkedin', 'fax', 'mobile']:
#             if not getattr(merged, field) and getattr(donor, field):
#                 setattr(merged, field, getattr(donor, field))

#         # Merge metadata
#         merged.source_context = f"{merged.source_context}\n{donor.source_context}"
#         merged.extraction_method = f"{merged.extraction_method},{donor.extraction_method}"

#         # Recalculate confidence
#         merged.confidence_score = max(
#             merged.confidence_score,
#             donor.confidence_score,
#             (merged.confidence_score + donor.confidence_score) / 2 + 0.1
#         )

#         return merged

#     def deduplicate(self, contacts: List[Contact], threshold: float = 0.7) -> List[Contact]:
#         """Deduplicate contacts using intelligent matching"""
#         if not contacts:
#             return []

#         # Build similarity matrix
#         n = len(contacts)
#         similarity_matrix = np.zeros((n, n))

#         for i in range(n):
#             for j in range(i + 1, n):
#                 sim = self.calculate_similarity(contacts[i], contacts[j])
#                 similarity_matrix[i][j] = sim
#                 similarity_matrix[j][i] = sim

#         # Find groups of similar contacts
#         merged_indices = set()
#         final_contacts = []

#         for i in range(n):
#             if i in merged_indices:
#                 continue

#             # Find all contacts similar to this one
#             similar_indices = [i]
#             for j in range(i + 1, n):
#                 if j not in merged_indices and similarity_matrix[i][j] >= threshold:
#                     similar_indices.append(j)
#                     merged_indices.add(j)

#             # Merge similar contacts
#             if len(similar_indices) == 1:
#                 final_contacts.append(contacts[i])
#             else:
#                 # Merge all similar contacts
#                 merged = contacts[similar_indices[0]]
#                 for idx in similar_indices[1:]:
#                     merged = self.merge_contacts(merged, contacts[idx])
#                 final_contacts.append(merged)

#         # Sort by confidence score
#         final_contacts.sort(key=lambda x: x.confidence_score, reverse=True)

#         return final_contacts

# class IntelligentContactExtractor:
#     """Main extraction pipeline with all components integrated"""

#     def __init__(self):
#         self.pdf_extractor = IntelligentPDFExtractor()
#         self.api_manager = SmartAPIManager()
#         self.validator = AdvancedValidator()
#         self.deduplicator = IntelligentDeduplicator()

#         # Load NLP model
#         try:
#             import spacy
#             self.nlp = spacy.load("en_core_web_sm")
#         except:
#             logger.warning("spaCy model not loaded. Some features may be limited.")
#             self.nlp = None

#         self.chunker = IntelligentTextChunker(self.nlp)

#     def process_pdf(self, pdf_path: str, progress_callback=None) -> List[Contact]:
#         """Main extraction pipeline"""
#         try:
#             # Step 1: Extract content using multiple methods
#             if progress_callback:
#                 progress_callback(0.1, "Extracting PDF content...")

#             extraction_results = self.pdf_extractor.extract_all_methods(pdf_path)

#             # Step 2: Analyze document structure
#             if progress_callback:
#                 progress_callback(0.2, "Analyzing document structure...")

#             doc_analysis = extraction_results.get('analysis', {})

#             # Step 3: Create intelligent chunks
#             if progress_callback:
#                 progress_callback(0.3, "Creating intelligent chunks...")

#             chunks = self.chunker.chunk_by_structure(
#                 extraction_results['text'],
#                 extraction_results['tables']
#             )

#             # Step 4: Extract contacts from each chunk
#             all_contacts = []
#             total_chunks = len(chunks)

#             for i, chunk in enumerate(chunks):
#                 if progress_callback:
#                     progress = 0.3 + (0.4 * (i / total_chunks))
#                     progress_callback(progress, f"Processing chunk {i+1}/{total_chunks}...")

#                 chunk_contacts = self.api_manager.extract_contacts_from_chunk(
#                     chunk,
#                     doc_analysis
#                 )

#                 # Validate and enhance each contact
#                 for contact in chunk_contacts:
#                     # Validate and normalize fields
#                     if contact.email:
#                         valid, conf = self.validator.validate_email(contact.email)
#                         if not valid:
#                             contact.email = ""

#                     if contact.phone:
#                         valid, conf, normalized = self.validator.validate_phone(contact.phone)
#                         if valid:
#                             contact.phone = normalized
#                         else:
#                             contact.phone = ""

#                     if contact.website:
#                         valid, conf, normalized = self.validator.validate_website(contact.website)
#                         if valid:
#                             contact.website = normalized

#                     # Calculate overall confidence
#                     contact.confidence_score = self.validator.calculate_overall_confidence(contact)

#                     # Only keep valid contacts
#                     if contact.confidence_score > 0.3 and (contact.email or contact.phone):
#                         all_contacts.append(contact)

#             # Step 5: Deduplicate contacts
#             if progress_callback:
#                 progress_callback(0.8, "Deduplicating contacts...")

#             unique_contacts = self.deduplicator.deduplicate(all_contacts)

#             # Step 6: Final filtering
#             if progress_callback:
#                 progress_callback(0.9, "Finalizing results...")

#             final_contacts = [
#                 contact for contact in unique_contacts
#                 if contact.confidence_score >= 0.5
#             ]

#             if progress_callback:
#                 progress_callback(1.0, f"Extracted {len(final_contacts)} contacts!")

#             return final_contacts

#         except Exception as e:
#             logger.error(f"Extraction pipeline error: {e}")
#             logger.error(traceback.format_exc())
#             raise

# def main():
#     """Enhanced Streamlit application"""
#     st.set_page_config(
#         page_title="Intelligent Contact Extractor",
#         page_icon="🤖",
#         layout="wide"
#     )

#     st.title("🤖 Intelligent Contact Extractor")
#     st.markdown("**AI-Powered Contact Extraction with Advanced Intelligence**")

#     # Initialize session state
#     if 'extracted_contacts' not in st.session_state:
#         st.session_state.extracted_contacts = []

#     # Sidebar for configuration
#     with st.sidebar:
#         st.header("🔧 Configuration")

#         # API key inputs
#         gemini_key = st.text_input(
#             "Gemini API Key",
#             value=GEMINI_API_KEY,
#             type="password",
#             help="Get your free API key from Google AI Studio"
#         )
#         if gemini_key:
#             os.environ['GEMINI_API_KEY'] = gemini_key
#             genai.configure(api_key=gemini_key)

#         deepseek_key = st.text_input(
#             "DeepSeek API Key (Backup)",
#             value=DEEPSEEK_API_KEY,
#             type="password"
#         )
#         if deepseek_key:
#             os.environ['DEEPSEEK_API_KEY'] = deepseek_key

#         st.markdown("---")

#         # Settings
#         st.subheader("Extraction Settings")
#         min_confidence = st.slider(
#             "Minimum Confidence Score",
#             0.0, 1.0, 0.5, 0.1,
#             help="Contacts below this confidence will be filtered out"
#         )

#         dedup_threshold = st.slider(
#             "Deduplication Threshold",
#             0.5, 1.0, 0.7, 0.05,
#             help="Similarity threshold for merging duplicate contacts"
#         )

#         st.markdown("---")

#         # API Status
#         st.subheader("🔌 API Status")

#         api_status = {
#             "Gemini": "✅ Connected" if gemini_key else "❌ Not configured",
#             "DeepSeek": "✅ Connected" if deepseek_key else "❌ Not configured",
#         }

#         for api, status in api_status.items():
#             st.write(f"{api}: {status}")

#         st.markdown("---")

#         # Info
#         st.info(
#             "This system uses advanced AI to intelligently extract contacts "
#             "from any PDF format - tables, directories, business cards, or "
#             "unstructured text."
#         )

#     # Main interface
#     col1, col2 = st.columns([2, 1])

#     with col1:
#         st.header("📄 Upload Document")
#         uploaded_file = st.file_uploader(
#             "Upload PDF (Any format - we'll figure it out!)",
#             type="pdf",
#             help="Supports: Business directories, contact lists, tables, "
#                  "business cards, membership rosters, and more"
#         )

#     with col2:
#         if uploaded_file:
#             st.success(f"✅ Uploaded: {uploaded_file.name}")
#             st.write(f"Size: {uploaded_file.size / 1024 / 1024:.2f} MB")

#     if uploaded_file is not None:
#         # Save uploaded file
#         temp_path = f"/tmp/{uploaded_file.name}"
#         with open(temp_path, "wb") as f:
#             f.write(uploaded_file.getbuffer())

#         # Extract contacts button
#         if st.button("🚀 Extract Contacts Intelligently", type="primary"):
#             # Create progress placeholder
#             progress_bar = st.progress(0)
#             status_text = st.empty()

#             def update_progress(progress, message):
#                 progress_bar.progress(progress)
#                 status_text.text(message)

#             try:
#                 # Initialize extractor
#                 extractor = IntelligentContactExtractor()

#                 # Extract contacts
#                 contacts = extractor.process_pdf(temp_path, update_progress)

#                 # Store in session state
#                 st.session_state.extracted_contacts = contacts

#                 # Clear progress
#                 progress_bar.empty()
#                 status_text.empty()

#                 if contacts:
#                     st.success(f"🎉 Successfully extracted {len(contacts)} high-quality contacts!")
#                 else:
#                     st.warning("No contacts found. The document might not contain contact information.")

#             except Exception as e:
#                 st.error(f"Error during extraction: {str(e)}")
#                 logger.error(traceback.format_exc())

#         # Display results if available
#         if st.session_state.extracted_contacts:
#             contacts = st.session_state.extracted_contacts

#             # Filter by confidence
#             filtered_contacts = [
#                 c for c in contacts
#                 if c.confidence_score >= min_confidence
#             ]

#             st.header("📋 Extracted Contacts")

#             # Summary metrics
#             col1, col2, col3, col4, col5 = st.columns(5)
#             with col1:
#                 st.metric("Total Contacts", len(filtered_contacts))
#             with col2:
#                 avg_confidence = sum(c.confidence_score for c in filtered_contacts) / len(filtered_contacts)
#                 st.metric("Avg Confidence", f"{avg_confidence:.2f}")
#             with col3:
#                 complete_contacts = sum(1 for c in filtered_contacts if c.email and c.phone)
#                 st.metric("Complete Contacts", complete_contacts)
#             with col4:
#                 business_contacts = sum(1 for c in filtered_contacts if c.business_name)
#                 st.metric("With Business Info", business_contacts)
#             with col5:
#                 unique_methods = len(set(c.extraction_method for c in filtered_contacts))
#                 st.metric("Extraction Methods", unique_methods)

#             # Display options
#             display_format = st.radio(
#                 "Display Format",
#                 ["Interactive Table", "Detailed Cards", "JSON View"],
#                 horizontal=True
#             )

#             if display_format == "Interactive Table":
#                 # Convert to DataFrame for display
#                 df_data = []
#                 for contact in filtered_contacts:
#                     df_data.append({
#                         "Name": contact.name,
#                         "Business": contact.business_name,
#                         "Email": contact.email,
#                         "Phone": contact.phone,
#                         "Title": contact.title,
#                         "Website": contact.website,
#                         "Confidence": f"{contact.confidence_score:.2f}",
#                         "Method": contact.extraction_method.split('_')[0]
#                     })

#                 df = pd.DataFrame(df_data)

#                 # Editable dataframe
#                 edited_df = st.data_editor(
#                     df,
#                     use_container_width=True,
#                     num_rows="dynamic",
#                     column_config={
#                         "Confidence": st.column_config.ProgressColumn(
#                             "Confidence",
#                             help="AI confidence in this contact",
#                             format="%.2f",
#                             min_value=0,
#                             max_value=1,
#                         ),
#                     }
#                 )

#             elif display_format == "Detailed Cards":
#                 # Display as expandable cards
#                 for i, contact in enumerate(filtered_contacts):
#                     with st.expander(
#                         f"📇 {contact.name or 'Contact ' + str(i+1)} "
#                         f"(Confidence: {contact.confidence_score:.2f})"
#                     ):
#                         col1, col2 = st.columns(2)

#                         with col1:
#                             st.write("**Personal Information**")
#                             st.write(f"Name: {contact.name}")
#                             st.write(f"Title: {contact.title}")
#                             st.write(f"Email: {contact.email}")
#                             st.write(f"Phone: {contact.phone}")
#                             if contact.mobile:
#                                 st.write(f"Mobile: {contact.mobile}")
#                             if contact.fax:
#                                 st.write(f"Fax: {contact.fax}")

#                         with col2:
#                             st.write("**Business Information**")
#                             st.write(f"Company: {contact.business_name}")
#                             st.write(f"Department: {contact.department}")
#                             st.write(f"Website: {contact.website}")
#                             st.write(f"Address: {contact.address}")
#                             if contact.linkedin:
#                                 st.write(f"LinkedIn: {contact.linkedin}")

#                         st.write("**Metadata**")
#                         st.write(f"Extraction Method: {contact.extraction_method}")
#                         st.write(f"Page: {contact.page_number}")

#                         if contact.source_context:
#                             st.text_area(
#                                 "Source Context",
#                                 value=contact.source_context,
#                                 height=50,
#                                 disabled=True
#                             )

#             else:  # JSON View
#                 # Display as JSON
#                 json_data = [contact.to_dict() for contact in filtered_contacts]
#                 st.json(json_data)

#             # Export options
#             st.header("💾 Export Data")

#             col1, col2, col3, col4 = st.columns(4)

#             with col1:
#                 # CSV export
#                 csv_data = pd.DataFrame([c.to_dict() for c in filtered_contacts]).to_csv(index=False)
#                 st.download_button(
#                     label="📊 Download CSV",
#                     data=csv_data,
#                     file_name=f"contacts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
#                     mime="text/csv"
#                 )

#             with col2:
#                 # JSON export
#                 json_data = json.dumps([c.to_dict() for c in filtered_contacts], indent=2)
#                 st.download_button(
#                     label="📥 Download JSON",
#                     data=json_data,
#                     file_name=f"contacts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
#                     mime="application/json"
#                 )

#             with col3:
#                 # Excel export
#                 df = pd.DataFrame([c.to_dict() for c in filtered_contacts])
#                 excel_buffer = pd.ExcelWriter('buffer.xlsx', engine='xlsxwriter')
#                 df.to_excel(excel_buffer, index=False)
#                 excel_data = excel_buffer.to_bytes()

#                 st.download_button(
#                     label="📑 Download Excel",
#                     data=excel_data,
#                     file_name=f"contacts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx",
#                     mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
#                 )

#             with col4:
#                 # Email list export
#                 email_list = [c.email for c in filtered_contacts if c.email]
#                 email_data = "\n".join(email_list)

#                 st.download_button(
#                     label="📧 Email List",
#                     data=email_data,
#                     file_name=f"emails_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt",
#                     mime="text/plain"
#                 )

#         # Clean up temp file
#         if os.path.exists(temp_path):
#             os.remove(temp_path)

#     # Instructions
#     with st.expander("📖 How This Intelligent System Works"):
#         st.markdown("""
#         ### 🧠 Advanced Intelligence Features

#         **1. Multi-Method PDF Extraction**
#         - Uses 5+ different extraction methods (PDFPlumber, Camelot, Tabula, PDFMiner, PyMuPDF, OCR)
#         - Automatically selects best method based on document analysis
#         - Handles tables, scanned documents, and complex layouts

#         **2. Document Understanding**
#         - Analyzes document structure and type (table, directory, business cards, etc.)
#         - Adapts extraction strategy based on document characteristics
#         - Preserves context and relationships between data

#         **3. Intelligent Chunking**
#         - Context-aware chunking that doesn't break contact information
#         - Semantic understanding of text boundaries
#         - Table-aware processing

#         **4. Adaptive AI Prompting**
#         - Generates specialized prompts based on document type
#         - Uses document metadata to improve extraction accuracy
#         - Multi-API support with intelligent fallback

#         **5. Advanced Validation**
#         - Multi-level validation for emails, phones, websites
#         - International phone number support
#         - Confidence scoring based on data quality

#         **6. Smart Deduplication**
#         - ML-based similarity matching
#         - Intelligent contact merging
#         - Preserves best information from duplicates

#         ### 📄 Supported Document Types
#         - ✅ Business directories
#         - ✅ Membership lists
#         - ✅ Contact tables
#         - ✅ Business cards (scanned or digital)
#         - ✅ Company rosters
#         - ✅ Trade show attendee lists
#         - ✅ Professional associations
#         - ✅ Unstructured documents with embedded contacts

#         ### 🔧 Pro Tips
#         - The system automatically handles different formats - no configuration needed
#         - Higher resolution PDFs yield better results for scanned documents
#         - Confidence scores help identify the most reliable contacts
#         - Export in multiple formats for different use cases
#         """)

# if __name__ == "__main__":
#     main()
# '''

# with open('contact_extractor.py', 'w') as f:
#     f.write(script_content)

# print("✅ Script created successfully!")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import subprocess
import threading
import time
from pyngrok import ngrok

def run_streamlit():
    subprocess.run([
        "streamlit", "run",
        "/content/drive/MyDrive/Leads_Extraction_Data_UKPakTrade/enhanced_contact_extractor.py",
        "--server.port", "8501",
        "--server.headless", "true"
    ])

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# Wait for Streamlit to start
time.sleep(10)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🌐 Access your app at: {public_url}")
print(f"📱 Click the link above to open the Contact Extractor interface")


🌐 Access your app at: NgrokTunnel: "https://5b8dffdf1de8.ngrok-free.app" -> "http://localhost:8501"
📱 Click the link above to open the Contact Extractor interface
